# **MATH&ML-9. Математика ансамблевых методов**

# 1. Введение

✍ Представьте, что вы выполняете важный проект по машинному обучению. Конечно, вы можете делать всё самостоятельно, но если вы дополнительно узнаете мнение коллег, попросите ментора проверить ваши расчёты, найдёте как можно больше информации по вашей теме, то точно получите наилучший результат, ведь вы учтёте не только свои знания и выводы, но и информацию от других компетентных людей.

Составление прогнозов в машинном обучении может следовать такой же логике: один алгоритм часто даёт далёкий от желаемой точности прогноз, ведь у каждого метода есть свои ограничения, и в целом создание модели, которая строит очень близкие к реальности предсказания, — достаточно сложная задача. Однако если мы обучим на наших данных несколько моделей и обобщим результаты определённым образом, то сможем получить куда более точный результат.

Такой алгоритм решения задач машинного обучения называется **ансамблем моделей**.

> **Ансамбль моделей** — это метод, в котором несколько алгоритмов (или вариации одного и того же) обучаются на одних данных, а итоговый прогноз строится на основе всех полученных от моделей прогнозов.

![](https://lms.skillfactory.ru/assets/courseware/v1/c284641ce53a2d60f7bfedb59aa830a3/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md9_1_1.png)

Вы уже встречались с ансамблями, однако последние модули существенно обогатили ваши знания по математике, и теперь вы сможете изучить математическую основу и тонкости данных методов: особенности настройки параметров, различные библиотеки, нюансы применения тех или иных ансамблей. Это сделает вас более компетентными специалистами, глубоко понимающими суть применяемых методов.

### Задание 1.7

Найдите минимум функции \(x^{2}+x y-y+y^{3}\).

В качестве ответа введите координату по оси ординат. Ответ округлите до двух знаков после точки-разделителя.

In [1]:
from sympy import *

f, x, y = symbols('f x y')

f = x**2 + x*y - y + y**3

diff_x  = f.diff(x)

diff_y  = f.diff(y)

extr = solve([diff_x, diff_y], [x, y])

print('Точки экстремумов (x, y): ', extr)

print('Значение функции в первой точке: ', f.subs([(x, extr[0][0]), (y, extr[0][1])]))

print('Значение функции во второй точке: ', f.subs([(x, extr[1][0]), (y, extr[1][1])]))

Точки экстремумов (x, y):  [(-1/3, 2/3), (1/4, -1/2)]
Значение функции в первой точке:  -13/27
Значение функции во второй точке:  5/16


⭐ Вы справились с тестированием — поздравляем! Теперь вы точно можете быть уверены, что ваша подготовка достаточна для освоения данного модуля. Прежде чем приступить, давайте обозначим основные цели на следующие семь юнитов:

1. **Изучить основную терминологию, связанную с ансамблями моделей.**
Мы повторим уже знакомые вам основные понятия, которые используются в ансамблях, и познакомимся с рядом новых.
2. **Подробно разобрать реализацию разных видов ансамблей с математической и смысловой точек зрения.**
Мы намного подробнее, чем раньше, разберём все алгоритмы и изучим их математическую составляющую, чтобы лучше понимать принцип их работы, уметь более тонко их настраивать и за счёт этого добиваться наилучшей эффективности.
3. **Научиться решать задачи регрессии и классификации с использованием ансамблей моделей.**
Конечно же, полученные знания мы будем использовать для решения настоящих практических задач.
4. **Научиться настраивать параметры моделей для повышения прогностической точности.**
Мы рассмотрим параметры алгоритмов, которые можно регулировать, и разберёмся, как менять каждый из них для повышения точности предсказания.

Важно обозначить, что мы будем рассматривать три вида построения ансамблей: бэггинг, бустинг и стекинг. Для каждого из них мы изучим популярные вариации, программную реализацию и, конечно же, сравним их эффективность при решении задач.

В результате освоения этого модуля вы сможете применять ансамблевые методы для решения задач машинного обучения. Вы не просто будете знать плюсы и минусы ансамблевых методов и то, какие из них уместны в том или ином случае, но также будете понимать их суть и математическую составляющую.

Итак, вперёд к ансамблям! →

# 2. Ансамбли моделей. Бутстреппинг. Бэггинг

✍ В этом юните вы углубите свои знания ансамблей алгоритмов. Вы познакомитесь с тем, как формируются бутстреп-выборки, а также досконально изучите принцип **бэггинга** — самого простого варианта ансамблей.

В основе бэггинга лежит статистический метод, который называется **бутстрепом (bootstrap)**. Идея бутстрепа заключается в генерации выборок размера n из исходного датасета размера N путём случайного выбора элементов с повторениями в каждом из наблюдений.

**Рассмотрим идею бутстрепа на элементарном примере.**

Пусть у нас есть выборка из 12 клиентов компании: у каждого из них есть свой ID (от 1 до 12) и какие-то характеристики. Мы можем создавать из данной выборки множество различных новых выборок клиентов с новым количеством человек (в данном случае представлены выборки из пяти человек). При этом информацию про одного и того же клиента можно использовать повторно.

![](https://lms.skillfactory.ru/assets/courseware/v1/95929bfe488e7dd592014067919c91ad/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md9_2_1.png)

Это намного проще, чем находить новые выборки. По сути, мы собираем данные лишь единожды, а затем на их основе генерируем много выборок для обучения моделей. Это экономит огромные объёмы ресурсов и времени.

При некотором приближении можно считать, что получающиеся выборки являются независимыми и репрезентативными — **это важное допущение**.

Выборки можно назвать **независимыми**, если результаты испытаний и измерения, осуществляемые для одной выборки, никак не влияют на результаты, получаемые на другой выборке.

**Репрезентативность** заключается в соответствии характеристик выборки всей генеральной совокупности.

К примеру, если мы хотим исследовать мнение всех женщин России по какому-то вопросу, то все женщины России — это **генеральная совокупность**.

**Репрезентативная выборка** — это такая группа женщин, для которой основные характеристики соответствуют характеристикам для генеральной совокупности. Допустим, если среди всех российских женщин 60 % имеют детей, а 40 % — не имеют, то соотношение в выборке должно быть таким же.

Бутстреп-выборки часто используются для оценки различных статистических показателей, например разброса или доверительного интервала.

Если вычислять статистические оценки на нескольких независимых выборках, то мы можем, например, оценить их математическое ожидание или разброс. Приведём пример того, как это происходит с точки зрения математики.

Допустим, у нас есть некоторая выборка $x=(5,1,3,6,4)$, и мы хотим оценить для неё математическое ожидание. Например, это может быть выборка количества товаров, которые приобретали покупатели нашего магазина, и мы хотим найти ожидаемое количество товаров, которое купит случайный клиент.

Конечно, мы без проблем можем его вычислить:

$$E(x) =E(5,1,3,6,4) = \frac{1}{5} \cdot 5 + \frac{1}{5} \cdot1 + \frac{1}{5} \cdot3 + \frac{1}{5} \cdot6 + \frac{1}{5} \cdot4 = 1+0.2 +0.6 + 1.2 + 0.8 = 3.8$$

Однако это значение лишь на одной выборке, а мы хотели бы вычислить эту статистическую оценку на нескольких выборках и затем проанализировать разброс оценок.

Создаём несколько выборок с помощью бутстрепа и на каждой оцениваем математическое ожидание:

![](https://lms.skillfactory.ru/assets/courseware/v1/880fd8813d3d070f2434c836167af820/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md9_2_2.png)

Получили ряд значений:

$4.4, 3.8, 4.8, 4.0, 3.4, 4.2, 5.2$

Теперь давайте найдём дисперсию для этого ряда. Мы с вами делали это в модуле по теории вероятностей — самое время вспомнить!

### Задание 2.2

Вычислите дисперсию для этого ряда. Результат округлите до трёх знаков после точки-разделителя.

In [2]:
from statistics import pvariance

pvariance([4.4, 3.8, 4.8, 4.0, 3.4, 4.2, 5.2]).__round__(3)

0.317

Таким образом, мы понимаем, что если мы будем создавать различные новые выборки и вычислять для них средние значения, то для полученных значений дисперсия будет равна найденному вами выше значению. **Заметьте: мы узнали это, не собирая никаких новых данных.**

Формализуем только что проделанные действия математически:

1. Генерируем выборки. Необходимо создавать упорядоченные множества элементов, которые мы выбираем с возвратом из некоторого имеющегося у нас множества:

$$\left\{X_{1}, \ldots, X_{N}\right\}$$

2. Повторяем несколько раз процедуру генерации выборки:

$$X_{b}^{*}=\left(X_{b 1}^{*}, \ldots, X_{b N}^{*}\right), \ где \ 1 \leqslant b \leqslant B$$

3. Считаем интересующую нас статистику по каждой выборке:

$$T_{1}^{*}=T\left(X_{1}^{*}\right), \ldots, T_{B}^{*}=T\left(X_{B}^{*}\right)$$

4. Получаем бутстрепную оценку для интересующей нас статистики по этой выборке статистик. Например, для дисперсии она будет вычисляться так:
Отлично, мы разобрались с тем, что такое метод бутстрепа. Запомните его идею — совсем скоро она пригодится вам для понимания алгоритма бэггинга.

$$\widehat{D}_{\text {boot }}=\frac{1}{B} \sum_{b=1}^{B} T_{b}^{* 2}-\left(\frac{1}{B} \sum_{b=1}^{B} T_{b}^{*}\right)^{2}$$

Отлично, мы разобрались с тем, что такое метод бутстрепа. Запомните его идею — совсем скоро она пригодится вам для понимания алгоритма бэггинга.

## BIAS И VARIANCE

Прежде чем перейти непосредственно к ансамблям моделей, нам необходимо повторить bias-variance decomposition, или, как его называют по-русски, «разложение ошибки на смещение и разброс». Оно очень полезно для анализа ансамблей моделей.

> **Смещение** — это разница между математическим ожиданием для прогноза и реальным значением:
>
> $$Bias[\hat{f}(x)] = E[\hat{f}(x)]-y$$

Здесь:

* $E[\hat{f}(x)]$ — математическое ожидание для прогноза,
* $y$ — реальное значение функции.

**Смысл смещения** — способность получить лучшую среди всех возможных моделей, то есть максимально точные прогнозы.

**Алгоритмы со стабильно маленьким смещением:**
* KNN (n = 1);
* метод опорных векторов;
* деревья решений с большой глубиной.

**Алгоритмы с большим смещением:**
* линейная регрессия;
* логистическая регрессия;
* деревья решений с маленькой глубиной.


Также значение смещения часто называют **ошибкой смещения** или **ошибкой из-за смещения**.

Если у модели большое смещение, это значит, что ошибка будет достаточно велика из-за слишком сильного упрощения модели.

> **Разброс** — это величина разницы в результатах обучения модели на разных выборках:
>
> $$\operatorname{Var}[\hat{f}(x)]=\mathrm{E}\left[\left(\mathrm{E}[\hat{f}(x)]-\hat{f}(x)\right)^{2}\right]$$

**Примечание**. С математической точки зрения разброс модели определяется как математическое ожидание квадрата разницы ожидаемого прогноза и реализованного прогноза модели.

Разброс характеризует устойчивость модели к изменениям в обучающей выборке:

* Если результат сильно зависит от того, какие объекты присутствуют в выборке, разброс будет большим.
* Если алгоритм работает стабильно вне зависимости от особенностей выборки, разброс будет маленьким.



**Алгоритмы, показывающие маленький разброс:**
* линейная регрессия;
* логистическая регрессия;
* деревья решений с маленькой глубиной.

**Алгоритмы, показывающие большой разброс:**
* деревья решений с большой глубиной;
* KNN;
* метод опорных векторов.


Давайте рассмотрим разложение на смещение и разброс для линейной регрессии.

Пусть мы хотим предсказать значение y по значениям вектора $x$. Тогда зависимость $y$ от $x$ можно записать следующим образом:

$$y=f(x)+\varepsilon$$

В качестве $f(x)$ здесь выступает истинная зависимость ответов $y$ от характеристик объекта $x$ — мы её не знаем и пытаемся предсказать с помощью модели. Предсказания обозначим как $\hat{f}(x)$. Символом $\varepsilon$ обозначается случайная ошибка. Предполагается, что её математическое ожидание равно нулю — это просто шум.

Тогда давайте выразим ошибку для какого-то значения $х$. Она будет равняться математическому ожиданию для квадрата разницы между реальным и предсказанным значениями. По сути, это просто среднеквадратичная ошибка, записанная в немного иной форме:

$$\operatorname{Err}(x)=E\left[(y-\hat{f}(x))^{2}\right]$$

Также мы можем разложить среднеквадратичную ошибку следующим образом:

$$\operatorname{Err}(x)=(E[\hat{f}(x)]-y)^{2}+E\left[(\hat{f}(x)-E[\hat{f}(x)])^{2}\right]+\sigma_{\varepsilon}^{2}$$

$$\operatorname{Err}(x)=\operatorname{Bias}^{2}+Variance+Irreducible Error$$



### Доказательство равенства

Для начала представим $y$ как сумму значения функции $f$ и ошибки (вместо $f(x)$ будем далее для краткости писать просто $f$):

$$\mathrm{E}\left[(y-\hat{f})^{2}\right]=\mathrm{E}\left[(f+\varepsilon-\hat{f})^{2}\right]$$

Теперь в выражение, от которого мы ищем математическое ожидание, добавим математическое ожидание предсказанной функции и вычтем его же (это нужно для того, чтобы далее мы смогли выразить необходимые нам величины). Разумеется, если мы прибавляем какую-то величину, а потом её вычитаем, результат остаётся тем же.

$$=\mathrm{E}\left[(f+\varepsilon-\hat{f}+\mathrm{E}[\hat{f}]-\mathrm{E}[\hat{f}])^{2}\right]$$

Далее раскроем скобки, то есть возведём в квадрат сумму трёх слагаемых:

* $f-\mathrm{E}[\hat{f}]$,

* $\varepsilon$,

* $\mathrm{E}[\hat{f}]-\hat{f}$.

**Примечание**. Для разложения пользуемся формулой из алгебры:

$$(a+b+c)^{2}=a^{2}+b^{2}+c^{2}+2 a b+2 a c+2 b c$$



Получаем:

$=\mathrm{E}\left[(f-\mathrm{E}[\hat{f}])^{2}\right]+\mathrm{E}\left[\varepsilon^{2}\right]+\mathrm{E}\left[(\mathrm{E}[\hat{f}]-\hat{f})^{2}\right]+2 \mathrm{E}[(f-\mathrm{E}[\hat{f}]) \varepsilon]+2 \mathrm{E}[\varepsilon(\mathrm{E}[\hat{f}]-\hat{f})]+2 \mathrm{E}[(\mathrm{E}[\hat{f}]-\hat{f})(f-\mathrm{E}[\hat{f}])]$

$=(f-\mathrm{E}[\hat{f}])^{2}+\mathrm{E}\left[\varepsilon^{2}\right]+\mathrm{E}\left[(\mathrm{E}[\hat{f}]-\hat{f})^{2}\right]+2(f-\mathrm{E}[\hat{f}]) \mathrm{E}[\varepsilon]+2 \mathrm{E}[\varepsilon] \mathrm{E}[\mathrm{E}[\hat{f}]-\hat{f}]+2 \mathrm{E}[\mathrm{E}[\hat{f}]-\hat{f}](f-\mathrm{E}[\hat{f}])$

Сокращаем одинаковые слагаемые:

$=(f-\mathrm{E}[\hat{f}])^{2}+\mathrm{E}\left[\varepsilon^{2}\right]+\mathrm{E}\left[(\mathrm{E}[\hat{f}]-\hat{f})^{2}\right]$

Видим, что у нас есть дисперсия ошибки и дисперсия предсказания:

$=(f-\mathrm{E}[\hat{f}])^{2}+\operatorname{Var}[\varepsilon]+\operatorname{Var}[\hat{f}]$

$=\operatorname{Bias}[\hat{f}]^{2}+\sigma^{2}+\operatorname{Var}[\hat{f}]$

Итак, мы получили, что наша ошибка — это сумма смещения для квадрата прогноза, разброса и неустранимой случайной ошибки. Теперь мы понимаем, из чего состоит ошибка модели. Такое представление помогает нам исследовать с теоретической точки зрения некоторые алгоритмы машинного обучения и часто используется при изучении ансамблей.

Итак, мы получили, что наша ошибка — это сумма смещения для квадрата прогноза, разброса и неустранимой случайной ошибки. Теперь мы понимаем, из чего состоит ошибка модели. Такое представление помогает нам исследовать с теоретической точки зрения некоторые алгоритмы машинного обучения и часто используется при изучении ансамблей.

Рассмотрим иллюстрацию того, как сдвиг и разброс влияют на качество предсказания. На рисунке ниже вы видите цель (красный круг), в которую мы хотим попасть.

Есть четыре ситуации:

![](https://img.genial.ly/5fdc5ca1853b5759f6e69400/5040dd83-7a8e-444e-8fc0-a3445064b968.png)



В моделях машинного обучения принцип тот же, только в качестве центра мишени выступает минимально возможная ошибка.

Когда говорят про разложение на bias и variance, то часто упоминают некую **точку баланса**:

* Если модель очень простая, с маленьким количеством параметров, то, скорее всего, у неё будет очень большое смещение, но маленький разброс.
* Если модель очень сложная, со множеством параметров, у неё будет большой разброс и маленькое смещение.

Схематично эти зависимости можно изобразить следующим образом (это схема не для конкретной модели, а лишь иллюстрация тенденций):

![](https://lms.skillfactory.ru/assets/courseware/v1/eccb230bf722e42b7fbeeadddf60daae/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md9_2_4.png)

На графике выше по оси абсцисс отложена сложность модели ($Model \ Complexity$), а по оси ординат — ошибка ($Error$). Также изображены смещение ($Bias^2$), разброс ($Variance$) и ошибка ($Total \ Error$ — сумма смещения и разброса).

Как вы можете видеть, есть некоторая оптимальная точка, в которой разброс и смещение небольшие, а ошибка минимальна. Именно эта точка нас и интересует.

## БЭГГИНГ

Перейдём к понятию **бэггинг**.

При построении моделей всегда есть вероятность, что при обучении на других данных получились бы другие результаты. Для того чтобы нивелировать такую вероятность, можно использовать бэггинг.

Его идея состоит в том, что мы берём несколько независимых моделей и усредняем полученные по ним результаты. Таким образом мы получаем модель, имеющую меньший разброс, так как при её построении мы учли несколько моделей.

Как уже было сказано, в реальности получить много независимых выборок слишком сложно, так как найти столько данных обычно невозможно. Поэтому мы используем бутстреп-выборки, о которых говорили в начале юнита.

Важно отметить, что при бэггинге размер каждой бутстреп-выборки должен совпадать с размером исходной выборки.

Схематично процесс бэггинга можно представить следующим образом:

![](https://lms.skillfactory.ru/assets/courseware/v1/86233c8d821d894e9008415bda25a791/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/MATHML_md9_2_6.png)

Теперь давайте сформулируем и объясним эту идею математически.

Пусть у нас есть некоторая выборка, и мы с помощью бутстрепа генерируем из неё ещё $B$ выборок:

$$X_{1}, ..., X_{B}$$

После этого мы определяем много базовых алгоритмов (всего $B$ моделей — по числу выборок) и обучаем каждый базовый алгоритм $a_{i}(x)$ на своей выборке. После этого получаем итоговый результат:

$$a(x)=\frac{1}{B} \sum_{i=1}^{B} a_{i}(x)$$

* Если мы рассматриваем задачу классификации, то, по сути, модели «голосуют» за свой класс.

* Если мы рассматриваем задачу регрессии, то результат — просто среднее арифметическое прогнозов по всем моделям.

Теперь посмотрим, **насколько применение бэггинга поможет нам улучшить качество модели**.

Представим, что мы хотим использовать бэггинг для решения задачи регрессии и берём для этого $K$ базовых алгоритмов:

$$a_{1}(x), ..., a_{K}(x)$$

Конечно, если для каждого объекта известно значение целевой переменной, мы можем вычислить ошибку для каждой модели:

$$\varepsilon_{i}(x)=a_{i}(x)-y(x), i=1, \ldots, K$$

Также мы можем выразить математическое ожидание квадратичной ошибки (т. е., по сути, среднеквадратичную ошибку):

$$\mathbb{E}_{x}\left[\left(a_{i}(x)-y(x)\right)^{2}\right]=\mathbb{E}_{x}\left[\varepsilon_{i}^{2}(x)\right]$$

Ошибка здесь обозначается как .

Тогда, если мы усредним значение ошибки по всем моделям регрессии, то получим следующее:

$$\mathbb{E}_{1}=\frac{1}{n} \mathbb{E}_{x} \sum_{i=1}^{n} \varepsilon_{i}^{2}(x)$$

Будем считать, что математическое ожидание для всех ошибок равно нулю (то есть нет какого-то систематического смещения ошибок, и они случайные) и что все ошибки независимы друг от друга (то есть их коэффициент корреляции равен нулю):

$$\begin{aligned} \mathbb{E}_{x} \varepsilon_{i}(x) &=0 \\ \mathbb{E}_{x} \varepsilon_{i}(x) \varepsilon_{j}(x) &=0, i \neq j \end{aligned}$$

Теперь определим регрессионную функцию, которая будет брать ответы от всех обученных нами регрессионных моделей и просто усреднять их:

$$f(x)=\frac{1}{K} \sum_{i=1}^{K} a_{i}(x)$$

Теперь найдём для этой модели среднеквадратичную ошибку. Выразим её через математическое ожидание:

$$\mathbb{E}_{K}=\mathbb{E}_{x}\left(\frac{1}{K} \sum_{i=1}^{K} a_{i}(x)-y(x)\right)^{2}$$

Упростим часть внутри скобок, воспользовавшись введённым ранее утверждением о том, что $\mathbb{E}_{x}\left[\left(a_{i}(x)-y(x)\right)^{2}\right]=\mathbb{E}_{x}\left[\varepsilon_{i}^{2}(x)\right]$, и получим следующее:

$$\mathbb{E}_{x}\left[\left(a_{i}(x)-y(x)\right)^{2}\right]=\mathbb{E}_{x}\left[\varepsilon_{i}^{2}(x)\right]$$

$$=\mathbb{E}_{x}\left(\frac{1}{K} \sum_{i=1}^{K} \varepsilon_{i}\right)^{2}$$

После этого вынесем за скобку $\frac{1}{K}$, а также возведём внутреннюю часть скобки в квадрат:

$$=\frac{1}{K^{2}} \mathbb{E}_{x}\left(\sum_{i=1}^{K} \varepsilon_{i}^{2}(x)+\sum_{i \neq j} \varepsilon_{i}(x) \varepsilon_{j}(x)\right)$$

Ранее мы уже выяснили, что $\sum_{i \neq j} \varepsilon_{i}(x) \varepsilon_{j}(x)$ равняется нулю, так как все ошибки независимы. Сокращаем выражение и получаем в итоге:

$$\mathbb{E}_{K}=\frac{1}{K}\mathbb{E}_{1}$$

Получается, что путём усреднения предсказаний линейных регрессий мы смогли уменьшить среднеквадратичную ошибку в $K$ раз.

Однако тут важно отметить, что при решении прикладных задач эффект будет не таким выраженным, так как здесь мы использовали предположение о полной независимости ошибок, а в реальной жизни такое случается редко.

Также, чтобы иметь полное представление о характеристиках рассматриваемого алгоритма, давайте вспомним про разложение ошибки на смещение и разброс.

Доказано, что бэггинг не ухудшает показатель смещения модели, то есть смещение у ансамбля ровно такое же, как и у одного базового алгоритма.



### Доказательство

Выберем из нашей выборки $X$ бутстрепом $K$ раз выборку длиной $N$. Получим выборки $X_{1}, X_{2}, ..., X_{K}$. Обучим базовые модели $a(x)$ на данных подвыборках. Первую модель $a_{1}(x) = a(x,X_{1})$  обучим на первой выборке бутстрепа, вторую $a_{2}(x) = a(x,X_{2})$ — на второй и так далее. Выполнив данную процедуру $n$ раз, мы получим предсказание как усреднение по всем обученным на подвыборках моделях.

$$f(x,X) = \frac{1}{K} \sum_{i=1}^{K} (f_i (x))$$

Теперь рассмотрим изменение смещения (bias) и разброса (variance) ансамблирования по отношению к базовым моделям.

> **Смещение (bias)** есть не что иное, как математическое ожидание разности между истинными ответами y и предсказаниями ансамбля: 

![](https://lms.skillfactory.ru/assets/courseware/v1/0010efcbd8914782f96e32442230250f/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst-3-ml-8-51.png)

![](https://lms.skillfactory.ru/assets/courseware/v1/c742c46100695e837ba46566856184cb/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst-3-ml-8-52.png)

**Вывод**: смещение ансамбля равно смещению базовой модели ансамбля!

> Разброс (variance, обозначим далее как ) — это дисперсия ответов алгоритма:

![](https://lms.skillfactory.ru/assets/courseware/v1/3d239a00c5d9d71aec09651f109a150f/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst-3-ml-8-53.png)

![](https://lms.skillfactory.ru/assets/courseware/v1/c704097ba958bf98bb20e2aa7bbac971/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst-3-ml-8-54.png)

![](https://lms.skillfactory.ru/assets/courseware/v1/bc1b9ab29110bd4e40c3d2b1f947fbf1/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst-3-ml-8-55.png)

При условии некоррелированности базовых моделей, которая достигается за счёт обучения на бутстрепе, последнее слагаемое равно нулю. Итого:

$$var(f(x,X)) = \frac{1}{K^2} \sum_{i=1}^{K} E \left [(a_i (x,X)- E \left [a_i (x,X) \right ] )\right ]^2 = \frac{1}{K^2} \sum_{i=1}^{K} var(a(x, X_i))$$

Тогда, зная, что модели не коррелированы, получаем:

![](https://lms.skillfactory.ru/assets/courseware/v1/7c8d37f4978d49117ffa159ebaf2c527/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/dst-3-ml-8-57.png)

**Вывод**: разброс ансамбля уменьшается в $K$ раз по сравнению с разбросом базовой модели!

В общем случае разброс бэггинга будет выражаться следующим образом:

$$\frac{1}{K}\left(\operatorname{Var} a_{n}(x)\right)+\operatorname{Cov}\left(a_{n}(x), a_{m}(x)\right)$$

В данном выражении через $a_{n}(x)$ обозначен один из базовых алгоритмов, а за $a_{m}(x)$ — другой базовый алгоритм.

Из этого следует, что если модели (в данной формуле — базовые модели $a_{n}(x)$ и $a_{m}(x)$) независимы, то разброс для ансамбля типа бэггинг будет в $K$ раз меньше, чем разброс у отдельной модели.

**Резюмируем:**

* Бэггинг даёт уменьшение ошибки в $K$ раз по сравнению с одиночной моделью.
* Бэггинг не уменьшает смещение по сравнению с одиночной моделью.
* Бэггинг уменьшает разброс в $K$ раз по сравнению с одиночной моделью.

Важно отметить, что эти утверждения выведены и доказаны теоретически и будут выполняться на практике только в том случае, если между ошибками нулевая корреляция.

Как видим, бэггинг — очень эффективный и полезный алгоритм, так что есть смысл попрактиковаться с ним.

### Задание 2.7

Объёмная и содержательная практика у нас ещё впереди, но в качестве разминки давайте поработаем с уже известным вам датасетом о вине, который можно скачать [здесь](https://lms.skillfactory.ru/assets/courseware/v1/805b5c231251e174abb4fdbbd391adc3/asset-v1:SkillFactory+DST-3.0+28FEB2021+type@asset+block/wineQualityReds.zip).

Ранее вы обучали на данных только один алгоритм, а теперь мы попробуем сравнить несколько.

* Подготовьте данные к классификации. Условно разделите вино на хорошее и плохое. Хорошим вином будем называть то, параметр quality которого — 6 и более.
* Сравните несколько методов классификации: логистическую регрессию, дерево решений и бэггинг. Это позволит вам увидеть, как меняется качество в зависимости от выбора того или иного алгоритма.
* Разделите выборку на обучающую и тестовую в соотношении 70/30, в качестве значения параметра random_state возьмите число 42.
* Для начала обучите два классификатора: логистическую регрессию (с параметрами по умолчанию) и дерево решений (random_state = 42, максимальная глубина — 10).

1. Введите значение F1-score для классификатора, который показал наилучшее значение. Ответ округлите до трёх знаков после точки-разделителя.

In [37]:
import pandas as pd
from sklearn import metrics

data = pd.read_csv('data/wineQualityReds.zip')

data['quality'] = data['quality'].apply(lambda x: 1 if x >= 6 else 0)

data.head()

,Unnamed: 0,fixed.acidity,volatile.acidity,citric.acid,residual.sugar,chlorides,free.sulfur.dioxide,total.sulfur.dioxide,density,pH,sulphates,alcohol,quality
0,1,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0
1,2,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,0
2,3,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,0
3,4,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,1
4,5,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,0


In [38]:
from sklearn.model_selection import train_test_split

X = data[data.columns[:-1]]
y = data[data.columns[-1]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [65]:
from sklearn import linear_model

lr = linear_model.LogisticRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)
print('LogisticRegression F1 score: {:.3f}'.format(metrics.f1_score(y_test, y_pred)))

from sklearn import tree

dt = tree.DecisionTreeClassifier(
    random_state=42,
    max_depth=10
)
dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
print('DecisionTreeClassifier F1 score: {:.3f}'.format(metrics.f1_score(y_test, y_pred)))

LogisticRegression F1 score: 0.739
DecisionTreeClassifier F1 score: 0.760


c:\Users\Home\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### Задание 2.8

Обучите модель с использованием бэггинга (класс BaggingClassifier с random_state=42).

Возьмите из предыдущего задания алгоритм, показавший наилучшее качество, и укажите для него новое количество моделей — 1500. Вычислите новое значение F1-score.

In [66]:
from sklearn.ensemble import BaggingClassifier

bc = BaggingClassifier(
    base_estimator=dt,
    n_estimators=1500,
    random_state=42    
)

bc.fit(X_train, y_train)
y_pred = bc.predict(X_test)
print('BaggingClassifier F1 score: {:.3f}'.format(metrics.f1_score(y_test, y_pred)))

BaggingClassifier F1 score: 0.824


# 3. Случайный лес